In [9]:
#Packages importing
import ee
import geemap
import multiprocess as mp
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

ee.Authenticate()
ee.Initialize()


In [2]:
# Load the image collection.
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2021-01-01', '2023-12-31'))

In [3]:
#Fixed Parameters
# Visualization parameters.
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
    'gamma': [1.0, 1.0, 1.0]
}


In [4]:
def display_masked_image(bounding_box_coords, house_coords):
    # Initialize the Earth Engine library.
    ee.Initialize()
   
    
    # Unpack the bounding box coordinates
    min_lon, min_lat, max_lon, max_lat = bounding_box_coords
   
    # Create a bounding box for the area of interest.
    boundingBox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])
   
    # Define the coordinates of the area of interest.
    houseCoords = ee.Geometry.Polygon(house_coords)
   
    # Create an image composite or use the first image.
    firstImage = dataset.median()
   
    # Clip the image to the bounding box.
    clippedImage = firstImage.clip(boundingBox)
   
    # Create a mask from the houseCoords polygon.
    mask = ee.Image.constant(1).clip(houseCoords).mask()
   
    # Apply the mask to the clipped image.
    blackImage = ee.Image.constant([0, 0, 0, 255]).clip(boundingBox)
    maskedImage = clippedImage.updateMask(mask).blend(blackImage.updateMask(mask.Not()))
   
    # Initialize a map.
    m = geemap.Map()
    m.set_center((max_lon + min_lon) / 2, (max_lat + min_lat) / 2, 20)  # Center the map on the bounding box.
    m.add_layer(maskedImage, trueColorVis, 'True Color')
   
    visualizedImage = maskedImage.visualize(**trueColorVis)

    geemap.ee_export_image_to_drive(visualizedImage, description="testingexample"+str(index), folder="QTM550Team1Plottings", region=boundingBox, scale=0.05)
    index=index+1
    # Display the map.
    return m




In [60]:
#Write code that breaks down the dataset into 80 chunks and stores them

#Write a function who's argument is a number, 1,....,80

def run_function(index):
    #Splitdf   = gpd.read_file("/Users/gehaojie/Documents/GitHub/QTM-Project/Split/Split_GeoJson" + str(index) + ".geojson")
    Splitdf   = gpd.read_file("/Users/gehaojie/Documents/GitHub/QTM-Project/Split/sample50from1file.geojson")
    for i in range(len(Splitdf)):
        bounding_box_coords  = (Splitdf['Max Longitude'].iloc[i], Splitdf['Max Latitude'].iloc[i],Splitdf['Min Longitude'].iloc[i],Splitdf['Min Latitude'].iloc[i])  # min_lon, min_lat, max_lon, max_lat
        all_coords = []
        
        for polygon in Splitdf['geometry']:
            if polygon.geom_type == 'Polygon':
                coords = list(polygon.exterior.coords)
                all_coords.append(coords)
            elif polygon.geom_type == 'MultiPolygon':
                for part in polygon.geoms:  # Modified line
                    coords = list(part.exterior.coords)
                    all_coords.append(coords)
        house_coords = all_coords[i]
        display_masked_image(bounding_box_coords, house_coords)
        tasks = ee.batch.Task.list()
# Print details of all tasks, looking for your specific task by description.
        for task in tasks:
            if task.config['description'] == 'testingexample':
                print(task.status())




In [61]:
#1 doesn't matter for it, we need it when we are doing final loop for multiple files
run_function(1)

{'state': 'COMPLETED', 'description': 'testingexample', 'creation_timestamp_ms': 1707775109729, 'update_timestamp_ms': 1707775147882, 'start_timestamp_ms': 1707775110684, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/17WbUH57VNawEGcU-LGi5qkKP5P8pomwL'], 'attempt': 1, 'batch_eecu_usage_seconds': 3.9773154258728027, 'id': '4BDY5KSATYLO6RNPGG64OJGQ', 'name': 'projects/earthengine-legacy/operations/4BDY5KSATYLO6RNPGG64OJGQ'}
{'state': 'COMPLETED', 'description': 'testingexample', 'creation_timestamp_ms': 1707774841297, 'update_timestamp_ms': 1707774880505, 'start_timestamp_ms': 1707774842944, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/17WbUH57VNawEGcU-LGi5qkKP5P8pomwL'], 'attempt': 1, 'batch_eecu_usage_seconds': 4.601363182067871, 'id': '2WIJEZQVPYRORT7S765YC5GE', 'name': 'projects/earthengine-legacy/operations/2WIJEZQVPYRORT7S765YC5GE'}
{'state': 'COMPLETED', 'description': 'testingexample', 'creation_timesta

KeyboardInterrupt: 

This portion is the code to call up cores. Constructing

In [ ]:
list_indexes = list(range(200))
num_cpus = mp.cpu_count()

with mp.Pool(num_cpus) as pool:
    pool.map(run_function, list_indexes)


for i in range(80):
    read output files
    append results

